In [14]:
import fasttext
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [16]:
reddit_dataframe = pd.read_csv('/Users/krishnavyas/Desktop/NUS/text_mining/project/depression-detector/depression_detector/datasets/cleaned_reddit_all.csv')

In [17]:
cleaned_reddit = reddit_dataframe['cleanedreddit'].str.lower().tolist()

In [18]:
cleaned_reddit[0]

'i am not diagnosed as having anxiety but i have a few questionscan anxiety cause electric shocks in the face i have over the day usually slight tingling and in the evening these electric shocks in my face currently only once on the evening i also feel like my hair is very sensitive like there is constantly something in my hair or something brushed a part of it i tend to worry very much over health stuff and stuff in general also jumping to worse case ideas like i also tend to think about stuff of the past over and over again which i can not impact or change now like i made that wrong or could do that better or what does that person think and so on also some very trivial things the pain is not heavy rather annoying and it was worse around three months ago i dont feel stressed my work is very relaxed right now i am a programmer but my facial tingling shocks were significantly better in my two vacations in the last half year i also have a tmj disorder and it is stressed my out because it

In [20]:
# open file in write mode
with open(r'/Users/krishnavyas/Desktop/NUS/text_mining/project/depression-detector/depression_detector/datasets/reddit_data.txt', 'w') as fp:
    for i, item in tqdm_notebook(enumerate(cleaned_reddit)):
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

0it [00:00, ?it/s]

Done


In [21]:
model = fasttext.train_unsupervised('/Users/krishnavyas/Desktop/NUS/text_mining/project/depression-detector/depression_detector/datasets/reddit_data.txt',model='skipgram', minn=2, maxn=8, dim=300)

Read 18M words
Number of words:  26892
Number of labels: 0
Progress: 100.0% words/sec/thread:   33026 lr:  0.000000 avg.loss:  1.133248 ETA:   0h 0m 0s  2.0% words/sec/thread:   24218 lr:  0.049019 avg.loss:  2.157060 ETA:   0h21m 3s  29806 lr:  0.042631 avg.loss:  2.193373 ETA:   0h14m53s 26.8% words/sec/thread:   31024 lr:  0.036591 avg.loss:  2.191037 ETA:   0h12m16s% words/sec/thread:   31303 lr:  0.033230 avg.loss:  2.178180 ETA:   0h11m 2s31275 lr:  0.032984 avg.loss:  2.177818 ETA:   0h10m58s  31931 lr:  0.028354 avg.loss:  2.174217 ETA:   0h 9m14s 9m 5s 49.8% words/sec/thread:   31513 lr:  0.025093 avg.loss:  1.937730 ETA:   0h 8m17s  0h 8m 7s 51.2% words/sec/thread:   31301 lr:  0.024416 avg.loss:  1.896266 ETA:   0h 8m 7s 51.3% words/sec/thread:   31295 lr:  0.024334 avg.loss:  1.891304 ETA:   0h 8m 5s 51.8% words/sec/thread:   31285 lr:  0.024083 avg.loss:  1.876645 ETA:   0h 8m 0s 52.9% words/sec/thread:   31271 lr:  0.023574 avg.loss:  1.848086 ETA:   0h 7m50s avg.loss:  1

In [22]:
model.save_model("fasttext_reddit_300dim.bin")

In [23]:
import gensim

model = gensim.models.fasttext.load_facebook_vectors('/Users/krishnavyas/Desktop/NUS/text_mining/project/depression-detector/fasttext_reddit_300dim.bin')

In [2]:
model.word_vec

<bound method KeyedVectors.word_vec of <gensim.models.fasttext.FastTextKeyedVectors object at 0x7f9600816bf0>>

In [27]:
import numpy as np
model.key_to_index
vocab = []
embeds = []


for key, value in model.key_to_index.items():
    vocab.append(key)
    embeds.append(model.word_vec(key))

/var/folders/jc/qr6wvn915kg2y8zqz3ln90xr0000gn/T/ipykernel_1302/1510896883.py:9: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embeds.append(model.word_vec(key))


In [31]:
vocab_npa = np.array(vocab)
embs_npa = np.array(embeds)

In [32]:
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:10])

pad_emb_npa = np.zeros((1,embs_npa.shape[1])) 
unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True) 
#insert embeddings for pad and unk tokens at top of embs_npa.
embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))

['<pad>' '<unk>' 'i' 'to' 'and' 'my' 'the' 'a' 'of' 'have']


In [37]:
with open('vocab_npa.npy','wb') as f:
    np.save(f,vocab_npa)

with open('embs_npa.npy','wb') as f:
    np.save(f,embs_npa)